[参考記事](https://qiita.com/koji_mats/items/0533fbdeb9012a7e1494#appendix-prefect%E3%82%92k8s%E3%81%B8%E3%83%87%E3%83%97%E3%83%AD%E3%82%A4%E3%81%99%E3%82%8B%E6%89%8B%E9%A0%86)


# メモ

- flow や task にはユニークな名前を付ける必要がある(そうでないと警告発生)


# 1. Hello World!!


In [ ]:
# ver1での書き方
# 2系で実施しているので、実行エラー
# from prefect import Flow, task

# @task
# def get_name():
#     return "world"

# @task
# def hello(who):
#     print("hello, {}!".format(who))

# with Flow("HelloWorld") as flow:
#     who = get_name()
#     hello_world = hello(who)

# flow.run()

In [1]:
# 記事の内容を2系で書き換え
from prefect import task, flow


@task(name="get name task")
def get_name():
    return "World"


@task(name="hello task")
def hello(who):
    print(f"Hello, {who}!")


@flow(name="greeting flow")
def greeting_flow():
    who = get_name()
    hello(who)


if __name__ == "__main__":
    greeting_flow()

08:36:58.321 | INFO    | prefect.engine - Created flow run 'excellent-falcon' for flow 'greeting flow'

08:36:58.365 | INFO    | Flow run 'excellent-falcon' - Created task run 'get name task-0' for task 'get name task'

08:36:58.366 | INFO    | Flow run 'excellent-falcon' - Executing 'get name task-0' immediately...

08:36:58.408 | INFO    | Task run 'get name task-0' - Finished in state Completed()

08:36:58.422 | INFO    | Flow run 'excellent-falcon' - Created task run 'hello task-0' for task 'hello task'

08:36:58.423 | INFO    | Flow run 'excellent-falcon' - Executing 'hello task-0' immediately...

Hello, World!


08:36:58.467 | INFO    | Task run 'hello task-0' - Finished in state Completed()

08:36:58.481 | INFO    | Flow run 'excellent-falcon' - Finished in state Completed('All states completed.')

# 2. パラメータ化

- flow 実行時にパラメータを渡して、タスク内で使用する


In [4]:
# "name"という引数をflow側で用意して、taskに渡して実行する
from prefect import task, flow


@task(name="get name task")
def get_name(name):
    return name


@task(name="hello task")
def hello(who):
    print(f"Hello, {who}!")


@flow(name="greeting flow")
def greeting_flow(name):
    who = get_name(name)
    hello(who)


if __name__ == "__main__":
    greeting_flow("KOH")

/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'get name task' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_57820/1155948213.py:4' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'hello task' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_57820/1155948213.py:8' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/flows.py:244: UserWarning: A flow named 'greeting flow' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_57820/1155948213.py:12' conflicts wit

09:23:58.012 | INFO    | prefect.engine - Created flow run 'hidden-salmon' for flow 'greeting flow'

09:23:58.083 | INFO    | Flow run 'hidden-salmon' - Created task run 'get name task-0' for task 'get name task'

09:23:58.084 | INFO    | Flow run 'hidden-salmon' - Executing 'get name task-0' immediately...

09:23:58.130 | INFO    | Task run 'get name task-0' - Finished in state Completed()

09:23:58.141 | INFO    | Flow run 'hidden-salmon' - Created task run 'hello task-0' for task 'hello task'

09:23:58.142 | INFO    | Flow run 'hidden-salmon' - Executing 'hello task-0' immediately...

Hello, KOH!


09:23:58.187 | INFO    | Task run 'hello task-0' - Finished in state Completed()

09:23:58.204 | INFO    | Flow run 'hidden-salmon' - Finished in state Completed('All states completed.')

# 3. 明示的な依存関係の定義

- task1 -> task2 の順に実行したいようなケース
  - `タスクの返り値を次のタスクの引数に指定すること`でタスク間の依存関係を構成してきました。
  - 一方、`タスク間でデータの受け渡しが無い場合`でも依存関係を構成したいときは、下流タスクが依存する上流タスクを明示的に指定してあげる必要があります。


In [ ]:
# def my_flow():
#     x = task_1.submit()
#     # task 2 will wait for task_1 to complete
#     # y = task_2.submit(wait_for=[x])

In [3]:
from prefect import task, flow


@task(name="hello task")
def task1():
    print("Hello")


@task(name="world task")
def task2():
    print("World!!")


@flow(name="hello world flow")
def hello_world():
    x = task1.submit()
    y = task2.submit(wait_for=[x])


if __name__ == "__main__":
    hello_world()

08:47:27.636 | INFO    | prefect.engine - Created flow run 'laughing-millipede' for flow 'hello world flow'

08:47:27.676 | INFO    | Flow run 'laughing-millipede' - Created task run 'hello task-0' for task 'hello task'

08:47:27.677 | INFO    | Flow run 'laughing-millipede' - Submitted task run 'hello task-0' for execution.

08:47:27.691 | INFO    | Flow run 'laughing-millipede' - Created task run 'world task-0' for task 'world task'

08:47:27.692 | INFO    | Flow run 'laughing-millipede' - Submitted task run 'world task-0' for execution.

Hello


08:47:27.728 | INFO    | Task run 'hello task-0' - Finished in state Completed()

World!!


08:47:27.764 | INFO    | Task run 'world task-0' - Finished in state Completed()

08:47:27.780 | INFO    | Flow run 'laughing-millipede' - Finished in state Completed('All states completed.')

# 4. 動的 Flow

- タスクの map メソッドを使用して、実行時に上流タスクから渡された値に応じて動的にタスクを生成
  - map の引数にリスト(を返すタスクの結果)を渡すと、`リストの各要素ごとにタスクを実行`
  - map で実行されるタスクは可能であれば並列実行されるよう


In [ ]:
from random import randint
from prefect import task, flow


@task
def random_list():
    return [i for i in range(randint(1, 4))]


@task
def squared(x):
    return x**2


@task
def sum_up(l):
    print(f"Squared results: {l}")
    print(f"Sum: {sum(l)}")


@flow(name="dynamic flow")
def dynamic_flow():
    l = random_list()
    squared_res = squared.map(l)
    sum_res = sum_up(squared_res)
    print(f"squared_res：{squared_res}")
    print(f"squared_resのタイプ：{type(squared_res)}")


if __name__ == "__main__":
    dynamic_flow()

# 5. 条件分岐

- 1 系では、タスク（case、merge、ifelse、switch）は、prefect.tasks.control_flow からインポートして使用していた。
- 2 系では、Python ネイティブの if/else 文を使って条件ロジックを指定することができます。


In [ ]:
# from random import randint
# from prefect import task, Flow
# from prefect.tasks.control_flow.conditional import ifelse

# @task
# def check_even():
#     return randint(0, 10) % 2 == 0

# @task
# def even():
#     print("It's Even!!")

# @task
# def odd():
#     print("It's Odd!!")

# with Flow("IfElseFlow") as flow:
#     cond = check_even()
#     result = ifelse(cond, even, odd)

# flow.run()

In [ ]:
from random import randint
from prefect import task, flow


@task
def check_even():
    return randint(0, 10) % 2 == 0


@task
def even():
    print("It's Even!!")


@task
def odd():
    print("It's Odd!!")


@flow
def is_else_flow():
    cond = check_even.submit()
    print(f"cond: {cond}")
    print(f"cond.result(): {cond.result()}")
    if cond.result():
        even.submit(wait_for=[cond])
    else:
        odd.submit(wait_for=[cond])


if __name__ == "__main__":
    is_else_flow()

In [ ]:
from random import randint
from prefect import task, flow


@task
def check_even():
    return randint(0, 10) % 2 == 0


@task
def even():
    print("It's Even!!")


@task
def odd():
    print("It's Odd!!")


@flow
def is_else_flow():
    cond = check_even()
    if cond:
        even()
    else:
        odd()


if __name__ == "__main__":
    is_else_flow()

# 6. Flow の成功・失敗の条件

- 事前準備 -> 本処理 -> 後片付けのようなフローで、本処理の結果にかかわらず、後片付けをしたいケース


In [ ]:
import random
from prefect import task, flow


@task
def do_something_important():
    bool_ = random.random() > 0.5
    print(f"Is the number > 0.5? {bool_}")
    if bool_:
        raise ValueError("Non-deterministic error has occured.")


@task
def fail():
    print("Failure")


@task
def succeed():
    print("Success")


@task
def always_run():
    print("Running regardless of upstream task's state")


@flow(log_prints=True)
def main_flow():
    a = do_something_important.submit()
    # equivalent of all_failed or any_failed trigger in Prefect 1.0:
    if a.wait() == "Failed":
        fail.submit()  # the task is skipped if the condition is not true
    # equivalent of all_succesful trigger in Prefect 1.0
    else:
        succeed.submit()  # the task is skipped on upstream failure

    # equivalent of always_run or all_finished trigger in Prefect 1.0:
    always_run.submit()


if __name__ == "__main__":
    main_flow()

# 7. Task Runner
* [参考記事](https://zenn.dev/kiri1701/scraps/5f5d151467f1ee)
  * taskに指定した関数を通常実行 -> Task Runnerなし
    * taskは逐次実行
  * .submit()メソッドを使用 -> taskをTask Runnerに送信して、タスク実行の制御をすることができる
    * 逐次実行
    * 同時実行
    * 並列・分散実行: Dask, Ray

# 8. States
* flow内のあるtaskがコケたとき、次に実行するflowを実行するときに、最初のタスクから実行されるか、それともコケたタスクから実行されるか

In [25]:
import datetime
from random import randint
from prefect import task, flow


@task(refresh_cache=False)
def check_even():
    print(datetime.datetime.now())
    # return randint(0, 10) % 2 == 0
    try:
        x = 1 / int(input())
    except ZeroDivisionError:
        print("ゼロで割ることはできません。タスクは失敗しました。")
    finally:
        print("最初のタスクが終了しました")


@task
def even():
    print("It's Even!!")


@task
def odd():
    print("It's Odd!!")


@flow
def is_else_flow():
    cond = check_even.submit()
    # print(f"cond: {cond}")
    # print(f"cond.result(): {cond.result()}")
    if cond.result():
        even.submit(wait_for=[cond])
    else:
        odd.submit(wait_for=[cond])


if __name__ == "__main__":
    is_else_flow()

09:07:08.875 | INFO    | prefect.engine - Created flow run 'interesting-agouti' for flow 'is-else-flow'

09:07:08.916 | INFO    | Flow run 'interesting-agouti' - Created task run 'check_even-0' for task 'check_even'

09:07:08.917 | INFO    | Flow run 'interesting-agouti' - Submitted task run 'check_even-0' for execution.

2024-08-30 09:07:08.942374
最初のタスクが終了しました


09:07:10.502 | INFO    | Task run 'check_even-0' - Finished in state Completed()

09:07:10.524 | INFO    | Flow run 'interesting-agouti' - Created task run 'odd-0' for task 'odd'

09:07:10.525 | INFO    | Flow run 'interesting-agouti' - Submitted task run 'odd-0' for execution.

It's Odd!!


09:07:10.565 | INFO    | Task run 'odd-0' - Finished in state Completed()

09:07:10.580 | INFO    | Flow run 'interesting-agouti' - Finished in state Completed('All states completed.')